In [1]:
import fsspec
from datetime import datetime,timedelta
import json
from notebookutils import mssparkutils


StatementMeta(, a5c2a85e-777a-42c6-bcf1-f2bdbbc7e00e, 3, Finished, Available, Finished)

In [2]:
source = "abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/aq"
checkpoint_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/8d633764-c954-4f64-a6ff-b52dde76bd20/Files/checkpoint/checkpoint.json"
account_name = "4906b11e-1e59-4869-9321-062a4696a2db"
account_host = "onelake.dfs.fabric.microsoft.com"

fs = fsspec.filesystem(
    "abfss",
    account_name=account_name,
    account_host=account_host
)
with fs.open(checkpoint_path) as f:
    checkpoint=json.load(f)

StatementMeta(, a5c2a85e-777a-42c6-bcf1-f2bdbbc7e00e, 4, Finished, Available, Finished)

In [3]:
def latest_date():
    base_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/aq/hourly"
    years = [int(y.name) for y in mssparkutils.fs.ls(base_path)]
    year = max(years)
    months = [int(m.name) for m in mssparkutils.fs.ls(f"{base_path}/{year}")]
    month = max(months)
    days = [int(d.name) for d in mssparkutils.fs.ls(f"{base_path}/{year}/{str(month).zfill(2)}")]
    day = max(days)


    return (year, month, day, 0,0)


StatementMeta(, a5c2a85e-777a-42c6-bcf1-f2bdbbc7e00e, 5, Finished, Available, Finished)

In [4]:
to_silver = checkpoint.get("air_quality", datetime(2025, 12, 16, 0, 0))
latest_col=datetime(*latest_date())
to_silver = datetime.fromisoformat(to_silver) if isinstance(to_silver, str) else to_silver
 

StatementMeta(, a5c2a85e-777a-42c6-bcf1-f2bdbbc7e00e, 6, Finished, Available, Finished)

In [5]:
if to_silver<latest_col:
    checkpoint['air_quality']=latest_col.isoformat()
    with fs.open(checkpoint_path,'w') as f:
        json.dump(checkpoint,f,indent=2)

StatementMeta(, a5c2a85e-777a-42c6-bcf1-f2bdbbc7e00e, 7, Finished, Available, Finished)